## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-02-06-20-24 +0000,nyt,Florida Is Buying Plane Tickets for Unauthoriz...,https://www.nytimes.com/2025/08/01/us/florida-...
1,2025-08-02-06-15-10 +0000,nyt,Judges Keep Restrictions on Los Angeles Immigr...,https://www.nytimes.com/2025/08/02/us/los-ange...
2,2025-08-02-06-11-29 +0000,nyt,"Economy Updates: After a Weak Jobs Report, Tru...",https://www.nytimes.com/live/2025/07/31/busine...
3,2025-08-02-06-07-29 +0000,bbc,Vegetable boxes for low income households,https://www.bbc.com/news/articles/c0j9869qj87o...
4,2025-08-02-06-05-24 +0000,nyt,Trump Taps Climate Skeptics to Attack Science ...,https://www.nytimes.com/2025/07/31/climate/tru...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2336/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,69
211,new,21
177,tariffs,19
18,report,12
17,jobs,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
85,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...,147
94,2025-08-01-22-08-04 +0000,nypost,Trump fires Biden-appointed Bureau of Labor St...,https://nypost.com/2025/08/01/us-news/trump-fi...,133
115,2025-08-01-21-26-41 +0000,latimes,"A raging Trump digs in on his trade war, fires...",https://www.latimes.com/world-nation/story/202...,125
220,2025-08-01-15-15-26 +0000,wapo,Trump’s new tariffs will hit these major tradi...,https://www.washingtonpost.com/business/2025/0...,124
25,2025-08-02-02-59-07 +0000,bbc,Trump fires lead official on economic data as ...,https://www.bbc.com/news/articles/cvg3xrrzdr0o...,124


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
85,147,2025-08-01-22-33-00 +0000,wsj,President Trump fired the top Bureau of Labor ...,https://www.wsj.com/politics/policy/trump-orde...
128,53,2025-08-01-20-59-00 +0000,wsj,"Ghislaine Maxwell, the former associate of con...",https://www.wsj.com/politics/policy/epstein-as...
89,53,2025-08-01-22-27-28 +0000,nypost,Trump admin fires back at claims Clinton plan ...,https://nypost.com/2025/08/01/us-news/trump-ad...
214,51,2025-08-01-15-34-42 +0000,nypost,"Dow drops over 500 points as Trump tariffs, jo...",https://nypost.com/2025/08/01/business/dow-dro...
99,49,2025-08-01-21-57-00 +0000,wsj,The Trump administration is blocking funding f...,https://www.wsj.com/politics/policy/trump-cdc-...
46,49,2025-08-02-00-59-00 +0000,wsj,The New York Fed’s John Williams says the labo...,https://www.wsj.com/economy/central-banking/fe...
255,44,2025-08-01-11-25-01 +0000,nypost,Russian attack on Kyiv leaves at least 28 dead...,https://nypost.com/2025/08/01/world-news/russi...
209,44,2025-08-01-15-57-21 +0000,cbc,Viable trade deal wasn't on the table ahead of...,https://www.cbc.ca/news/politics/reaction-trum...
167,39,2025-08-01-18-48-42 +0000,nypost,Elon Musk’s Tesla ordered to pay $243M in dead...,https://nypost.com/2025/08/01/business/elon-mu...
184,36,2025-08-01-17-58-15 +0000,nypost,"Anti-Israel NYC Councilwoman Tiffany Caban, do...",https://nypost.com/2025/08/01/us-news/proteste...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
